In [ ]:
# Nov 27 2020
# Big Data Project 
# A small neural network

References:

https://keras.io/about/

https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

In [14]:
import csv 
import numpy as np
import random

import keras
from keras.models import Sequential
from keras.layers import Dense

In [25]:
filename = "cps3_reshape.csv"

data = [] 
  
# opening the CSV file 
with open(filename, mode ='r') as file:     
         
       # reading the CSV file 
       csvFile = csv.reader(file) 
  
       # displaying the contents of the CSV file 
       for line in csvFile: 
            data.append(line)

In [26]:
print(data[0][:39], data[0][42])
print(len(data))

['YEAR', 'SERIAL', 'MONTH', 'HWTFINL', 'CPSID', 'REGION', 'STATEFIP', 'METRO', 'FAMINC', 'HRHHID', 'HRHHID2', 'PERNUM', 'WTFINL', 'CPSIDP', 'RELATE', 'AGE', 'SEX', 'RACE', 'MARST', 'POPSTAT', 'VETSTAT', 'NCHILD', 'BPL', 'YRIMMIG', 'CITIZEN', 'NATIVITY', 'HISPAN', 'EMPSTAT', 'LABFORCE', 'OCC', 'IND', 'CLASSWKR', 'UHRSWORKT', 'WHYUNEMP', 'WKSTAT', 'PROFCERT', 'EDUC', 'SCHLCOLL', 'DIFFANY'] EMPSTAT2
47000


In [113]:
count = 0
for i in range(len(data)):
    if data[i][27]==data[i][42]:
        count += 1
print("There are", count, "observations where empstat=empstat2, so baseline accuracy is", count/len(data))
# (if using empstat (27) in input)

There are 39336 observations where empstat=empstat2, so baseline accuracy is 0.836936170212766


In [184]:
# these are the indices of the variables the network will use
#indexNet = [0, 1, 2, 5, 6, 7, 8, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
indexNet = [8, 15, 16, 17, 24]
#indexNet = [16, 17, 27]
#indexNet = [27]

In [185]:
# generate training and testing data

dataNet = data[1:]
random.shuffle(dataNet)

Xtrain = []
ytrain = []
Xtest = []
ytest = []

for obs in range(1, 40001):
    row = []
    for i in indexNet:
        row.append(int(dataNet[obs][i]))
    Xtrain.append(row)
    if dataNet[obs][42] in ['1', '10', '12']: # employed 
        ytrain.append([1])
    else:
        ytrain.append([0])
Xtrain = np.array(Xtrain)
ytrain = np.array(ytrain)

for obs in range(40001, 46999):
    row = []
    for i in indexNet:
        row.append(int(dataNet[obs][i]))
    Xtest.append(row)
    if dataNet[obs][42] in ['1', '10', '12']: # employed 
        ytest.append([1])
    else:
        ytest.append([0])
Xtest = np.array(Xtest)
ytest = np.array(ytest)

In [181]:
net = Sequential()
net.add(Dense(10, input_dim=len(indexNet), activation = 'relu'))
#net.add(Dense(20, activation='relu'))
#net.add(Dense(10, activation='relu'))
net.add(Dense(30, activation='relu'))
net.add(Dense(1, activation='sigmoid'))

net.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

net.fit(Xtrain, ytrain, epochs=20, batch_size=10)

Epoch 1/20
4000/4000 [==============================] - 7s 2ms/step - loss: 1.0613 - accuracy: 0.5604
Epoch 2/20
4000/4000 [==============================] - 7s 2ms/step - loss: 0.6157 - accuracy: 0.7134
Epoch 3/20
4000/4000 [==============================] - 6s 2ms/step - loss: 0.5719 - accuracy: 0.7430
Epoch 4/20
4000/4000 [==============================] - 6s 2ms/step - loss: 0.5393 - accuracy: 0.7539
Epoch 5/20
4000/4000 [==============================] - 6s 2ms/step - loss: 0.5038 - accuracy: 0.7660
Epoch 6/20
4000/4000 [==============================] - 6s 2ms/step - loss: 0.4825 - accuracy: 0.7778
Epoch 7/20
4000/4000 [==============================] - 7s 2ms/step - loss: 0.4655 - accuracy: 0.7881
Epoch 8/20
4000/4000 [==============================] - 7s 2ms/step - loss: 0.4533 - accuracy: 0.7904
Epoch 9/20
4000/4000 [==============================] - 7s 2ms/step - loss: 0.4480 - accuracy: 0.7935
Epoch 10/20
4000/4000 [==============================] - 7s 2ms/step - loss: 0.442

In [183]:
net.summary()

Model: "sequential_71"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_206 (Dense)            (None, 10)                60        
_________________________________________________________________
dense_207 (Dense)            (None, 30)                330       
_________________________________________________________________
dense_208 (Dense)            (None, 1)                 31        
Total params: 421
Trainable params: 421
Non-trainable params: 0
_________________________________________________________________


In [182]:
score = net.evaluate(Xtest, ytest, batch_size=100)

70/70 [==============================] - 0s 1ms/step - loss: 0.4248 - accuracy: 0.8031


### Notes: 

indexNet = [27], plus this gets about 90%. 

    net = Sequential()
    net.add(Dense(2, input_dim=len(indexNet), activation = 'relu'))
    net.add(Dense(2, activation='sigmoid')) 
    net.add(Dense(1, activation='sigmoid'))


With indexNet = [16, 27] (sex and empstat), this gets 90%. 

    net = Sequential()
    net.add(Dense(10, input_dim=len(indexNet), activation = 'relu'))
    net.add(Dense(1, activation='sigmoid'))
    
    net.compile(optimizer='sgd',
              loss='mse',
              metrics=['accuracy'])

    net.fit(Xtrain, ytrain, epochs=10, batch_size=10)

Updated per blog post with indexNet = [16, 17, 27] got 91%

    net.add(Dense(12, input_dim=len(indexNet), activation = 'relu'))
    net.add(Dense(8, activation='relu'))
    net.add(Dense(1, activation='sigmoid'))

    net.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    net.fit(Xtrain, ytrain, epochs=20, batch_size=10)
     